In [1]:
import torch
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from shapely import wkt
from tqdm import tqdm
from geopy.distance import great_circle
from sklearn.preprocessing import OneHotEncoder

from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset

from tensorboardX import SummaryWriter
from datetime import datetime

tqdm.pandas()

/var/essdata/IDU/venvs/common_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
class MigrationDataset(InMemoryDataset):
    def __init__(self, root, transform=None):
        super(MigrationDataset, self).__init__(root, transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        files = []
        for file in os.listdir(self.root):
            if file.endswith(".geojson") or file.endswith(".csv"):
                files.append(file)
        return files
        
    @property
    def processed_file_names(self):
        return ['migration_dataset']

    def download(self):
        pass
    
    def process(self):
        
        # read files in specified folder
        cities = gpd.read_file(os.path.join(self.root, "cities_aggregated.geojson")).set_index("city")
        responses = gpd.read_file(os.path.join(self.root, "responses_clustered.csv"))


        # extract cities features 
        cities_features = cities[[
            "population",
            "city_category", 
            "harsh_climate", 
            "ueqi_score", 
            "ueqi_residential", 
            "ueqi_street_networks", 
            "ueqi_green_spaces", 
            "ueqi_public_and_business_infrastructure", 
            "ueqi_social_and_leisure_infrastructure",
            "ueqi_citywide_space",
            "cvs_total",
            "vacancies_total",
            "factories_count"
            ]]

        # encode categorical features
        one_hot = OneHotEncoder()
        encoded_category = one_hot.fit_transform(np.expand_dims(cities["city_category"].to_numpy(), 1)).toarray()
        encoded_category_names = one_hot.get_feature_names_out(["category"])
        cities_features.loc[:, encoded_category_names] = encoded_category
        cities_features = cities_features.drop(["city_category"], axis=1)
        cities_features["harsh_climate"] = cities_features["harsh_climate"].astype(int)

        # form distance matrix
        DM = cities["geometry"].progress_apply(
            lambda p1: cities["geometry"].apply(
                lambda p2: great_circle(p1.coords[0], p2.coords[0]).km
                ))

        # form origin-destination matrix

        responses_counts = responses.groupby(["cluster_center_cv", "cluster_center_vacancy"])["id_candidate"].count()
        responses_cities = responses_counts.index.get_level_values(0).drop_duplicates()
        OD = pd.DataFrame(None, index=DM.columns, columns=DM.columns)
        OD = OD.progress_apply(
            lambda city: city.fillna(responses_counts[city.name]).fillna(0) 
            if city.name in responses_cities else city.fillna(0)
            )
        
        # transform data
        
        cities_num = len(OD)
        edge_index = [[], []]
        for i in range(cities_num):
            edge_index[0].extend([i for j in range(cities_num)])
            edge_index[1].extend([j for j in range(cities_num)])

        edge_index = torch.tensor(edge_index)
        y = torch.tensor(np.concatenate((OD.to_numpy())), dtype=torch.float32)
        edge_attr = torch.tensor(np.concatenate((DM.to_numpy())), dtype=torch.float32)
        x = torch.tensor(cities_features.to_numpy(), dtype=torch.float32)

        # exclude diagonal
        non_diagonal = edge_attr > 0
        edge_attr = edge_attr[non_diagonal]
        edge_index = edge_index[:, non_diagonal]
        y = y[non_diagonal]
        
        # create torch object          
        graph = Data(x=x,edge_index=edge_index, y=y, edge_attr=edge_attr)
        
        data_list = []
        data_list.append(graph)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

Data can be found here: https://disk.yandex.ru/client/disk/%D0%A2%D1%80%D1%83%D0%B4%D0%BE%D0%B2%D1%8B%D0%B5%20%D1%80%D0%B5%D1%81%D1%83%D1%80%D1%81%D1%8B/%D0%A4%D0%B0%D0%B9%D0%BB%D1%8B/ML_experiments/data

In [3]:
dataset = MigrationDataset("/var/essdata/IDU/other/mm_22/industrial-location/ml/data")

In [4]:
class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(FNN, self).__init__()

        self.num_layers = num_layers

        self.lins = nn.ModuleList()
        self.lins.append(nn.Linear(input_dim, hidden_dim))
        for _ in range(self.num_layers - 2):
            self.lins.append(nn.Linear(hidden_dim, hidden_dim))
        self.lins.append(nn.Linear(hidden_dim, 1))

        self.norm = nn.ModuleList()
        for l in range(self.num_layers):
            self.norm.append(nn.LayerNorm(hidden_dim))

        self.dropout = dropout

    def forward(self, data):

        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr.unsqueeze(-1)

        x_s = x[edge_index[0]]
        x_d = x[edge_index[1]]
        y = torch.cat((x_s, x_d, edge_weight), axis=1)
        # y = F.normalize(y)

        for i in range(self.num_layers - 1):
            y = self.lins[i](y) 
            y = nn.functional.leaky_relu(y)
            y = F.dropout(y, p=self.dropout, training=self.training)
            y = self.norm[i](y)

        y = self.lins[-1](y)
        y = torch.relu(y).squeeze()
        
        return y

In [5]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

def train_func(dataset, model, epochs, writer):

    optimize = torch.optim.Adam(list(model.parameters()),  lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimize, factor=0.9, min_lr=0.0005)

    # train
    for epoch in range(epochs + 1):

        optimize.zero_grad()
        model.train()
        y_hat = model(dataset)

        loss = F.mse_loss(y_hat, dataset.y)
        r2 = r2_loss(y_hat, dataset.y)

        loss.backward()
        optimize.step()
        scheduler.step(loss)

        t_metrics = {"train_loss": loss, "train_r2": r2}  
        for name, v_metric in t_metrics.items(): writer.add_scalar(name, v_metric, epoch)

        if epoch % 10 == 0:    
            print(
                "Epoch {}. TRAIN: loss {:.4f}, r2: {:.4f}. lr: {:.4f} ".format(
                    epoch, t_metrics["train_loss"], t_metrics["train_r2"], optimize.param_groups[0]["lr"]
                    )
                ) 

    return model


In [6]:
input_dim = dataset.data.x.shape[1] * 2 + 1
hidden_dim = 256
num_layers = 5
dropout = 0.2

model_v1 = FNN(input_dim, hidden_dim, num_layers, dropout)

In [7]:
datetime_now = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "./logs/" + "add_features_256_5_layers" + datetime_now
writer = SummaryWriter(logdir)

trained_model = train_func(dataset.data, model_v1, 5000, writer)

Epoch 0. TRAIN: loss 394.2522, r2: -0.0010. lr: 0.0010 
Epoch 10. TRAIN: loss 393.5278, r2: 0.0009. lr: 0.0010 
Epoch 20. TRAIN: loss 393.1065, r2: 0.0020. lr: 0.0010 
Epoch 30. TRAIN: loss 392.8091, r2: 0.0027. lr: 0.0010 
Epoch 40. TRAIN: loss 392.2228, r2: 0.0042. lr: 0.0010 
Epoch 50. TRAIN: loss 391.5364, r2: 0.0059. lr: 0.0010 
Epoch 60. TRAIN: loss 390.3962, r2: 0.0088. lr: 0.0010 
Epoch 70. TRAIN: loss 389.1128, r2: 0.0121. lr: 0.0010 
Epoch 80. TRAIN: loss 387.6410, r2: 0.0158. lr: 0.0010 
Epoch 90. TRAIN: loss 386.1702, r2: 0.0196. lr: 0.0010 
Epoch 100. TRAIN: loss 384.3513, r2: 0.0242. lr: 0.0010 
Epoch 110. TRAIN: loss 382.6587, r2: 0.0285. lr: 0.0010 
Epoch 120. TRAIN: loss 380.9814, r2: 0.0327. lr: 0.0010 
Epoch 130. TRAIN: loss 379.1213, r2: 0.0375. lr: 0.0010 
Epoch 140. TRAIN: loss 377.1573, r2: 0.0424. lr: 0.0010 
Epoch 150. TRAIN: loss 375.3437, r2: 0.0471. lr: 0.0010 
Epoch 160. TRAIN: loss 375.1427, r2: 0.0476. lr: 0.0010 
Epoch 170. TRAIN: loss 374.1987, r2: 0.05

In [8]:
torch.save(model_v1, "/var/essdata/IDU/other/mm_22/industrial-location/ml/model.pth")